## Homework

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and 
model management.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?


In [1]:
!pip install -q mlflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 1.33.0 requires cachetools<5.0,>=2.0.0, but you have cachetools 5.3.3 which is incompatible.


In [2]:
!mlflow --version

mlflow, version 2.13.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:


In [5]:
!python preprocess_data.py --raw_data_path  /data/ --dest_path ./output

Traceback (most recent call last):
  File "/workspaces/MLOps/02-experiment-tracking/homework/preprocess_data.py", line 83, in <module>
    run_data_prep()
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/click/core.py", line 754, in invoke
    return __callback(*args, **kwargs)
  File "/workspaces/MLOps/02-experiment-tracking/homework/preprocess_data.py", line 50, in run_data_prep
    df_train = read_dataframe(
  File "/workspaces/MLOps/02-experiment-tracking/homework/preprocess_data.py", line 15, in read_dataframe
    df = pd.read_parquet(filename)
  File "/home/codes